# Table of skills for contest year only

Generates a LaTeX table of skills for each forecast method over the year 2017-2018.

In [ ]:
# Autoreload packages that are modified
%load_ext autoreload
%autoreload 2

# Load relevant packages
import numpy as np
import pandas as pd
from sklearn import *
import sys
import subprocess
from datetime import datetime, timedelta
import netCDF4
import time
from functools import partial
import os
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

# Ensure correct working directory
if os.path.basename(os.getcwd()) == "visualize":
    # Navigate to base folder
    os.chdir(os.path.join("..",".."))

# Adds 'experiments' folder to path to load experiments_util
sys.path.insert(0, 'src/experiments')
# Load general utility functions
from experiments_util import *

In [ ]:
# Hardcoded CFSv2 and damped persistence skills from Forecast Rodeo website
cfs_skills = pd.DataFrame()
damped_skills = pd.DataFrame()
cfs_skills['temp34'] = [-0.7978,
                    0.2989,
                    -0.3726,
                    0.5662,
                    0.5328,
                    0.7058,
                    0.5949,
                    0.5267,
                    0.3401,
                    0.515,
                    0.1444,
                    -0.1483,
                    0.6801,
                    -0.3062,
                    -0.3513,
                    0.6054,
                    0.2942,
                    0.5234,
                    -0.2593,
                    -0.3961,
                    -0.0228,
                    0.0688,
                    0.5483,
                    0.2409,
                    -0.0004,
                    -0.3997]
damped_skills['temp34'] = [0.4769,
                    0.6917,
                    0.8352,
                    0.1874,
                    0.6714,
                    0.3597,
                    0.5874,
                    -0.2783,
                    0.6303,
                    0.0824,
                    -0.6947,
                    -0.673,
                    -0.4985,
                    0.3493,
                    0.5472,
                    -0.4511,
                    0.6685,
                    0.1707,
                    -0.1956,
                    0.162,
                    0.33,
                    -0.4629,
                    0.7816,
                    0.3484,
                    0.5287,
                    -0.0794]
cfs_skills['temp56'] = [-0.1323,
                    0.5448,
                    0.4797,
                    0.6381,
                    0.6267,
                    0.3783,
                    0.4252,
                    0.3508,
                    0.2325,
                    -0.5738,
                    0.3052,
                    -0.276,
                    -0.3346,
                    0.7556,
                    -0.1714,
                    0.7087,
                    -0.5043,
                    -0.3052,
                    0.4187,
                    0.7087,
                    -0.2003,
                    0.644,
                    0.7306,
                    -0.3918,
                    0.1144,
                    0.5267]
damped_skills['temp56'] = [0.4784,
                    0.6678,
                    -0.0776,
                    0.0118,
                    -0.7227,
                    -0.5953,
                    -0.148,
                    -0.4409,
                    -0.4391,
                    0.0752,
                    0.6576,
                    -0.5816,
                    -0.0548,
                    0.2567,
                    -0.284,
                    0.0872,
                    0.432,
                    -0.351,
                    -0.0753,
                    -0.4779,
                    0.4081,
                    -0.2283,
                    -0.8073,
                    -0.2139,
                    0.3333,
                    0.1084]
cfs_skills['precip34'] = [-0.5695,
                    0.1829,
                    -0.8206,
                    -0.4563,
                    -0.3214,
                    -0.2522,
                    0.3193,
                    0.4274,
                    0.3535,
                    0.0753,
                    -0.1971,
                    0.1716,
                    0.7384,
                    0.3792,
                    0.1159,
                    0.429,
                    0.2906,
                    -0.3039,
                    0.3695,
                    -0.2275,
                    0.5451,
                    -0.1536,
                    0.2771,
                    -0.0562,
                    0.3544,
                    0.1837]
damped_skills['precip34'] = [-0.4545,
                    0.0522,
                    -0.6204,
                    -0.3544,
                    -0.3825,
                    -0.2814,
                    0.313,
                    0.2458,
                    0.012,
                    -0.4314,
                    0.5441,
                    -0.1724,
                    -0.325,
                    -0.2688,
                    0.0517,
                    -0.51,
                    -0.3092,
                    -0.5708,
                    0.4499,
                    -0.101,
                    -0.1923,
                    0.1488,
                    -0.2481,
                    0.1411,
                    0.2605,
                    -0.7996]
cfs_skills['precip56'] = [0.1022,
                    -0.8059,
                    -0.4429,
                    -0.2975,
                    -0.2694,
                    0.3117,
                    0.4651,
                    0.0753,
                    -0.1953,
                    -0.3254,
                    -0.0151,
                    0.2191,
                    0.3895,
                    -0.563,
                    0.7058,
                    -0.3406,
                    0.0839,
                    -0.3738,
                    0.6004,
                    0.7295,
                    -0.2749,
                    0.0706,
                    -0.1196,
                    0.2814,
                    0.5098,
                    0.0692]
damped_skills['precip56'] = [0.051,
                    -0.6225,
                    -0.354,
                    -0.3875,
                    -0.2781,
                    0.3083,
                    0.2419,
                    0.0097,
                    -0.4253,
                    0.5401,
                    -0.1761,
                    -0.3249,
                    -0.2685,
                    0.04,
                    -0.5063,
                    -0.3022,
                    -0.5756,
                    0.4498,
                    -0.48,
                    -0.1943,
                    0.1487,
                    -0.2456,
                    0.1373,
                    0.2561,
                    -0.7983,
                    -0.4375]

In [ ]:
# Collate skills of backward stepwise, KNN regression, ensemble of backward stepwise and KNN regression,
# CFSv2, and damped persistence into a dataframe

all_skills = pd.DataFrame()
for gt_id in ["contest_tmp2m", "contest_precip"]:
    for target_horizon in ["34w", "56w"]:
        gt_str = "temp" if gt_id == "contest_tmp2m" else "precip"
        horizon_str = "34" if target_horizon == "34w" else "56"
        task_str = gt_str + horizon_str
        backward_file="results/skills/backward_stepwise/skill-{}-{}-margin56-mean.h5".format(gt_id, target_horizon)
        knn_file="results/skills/knn_target_regression/skill-{}-{}-days60-early{}.h5".format(gt_id, target_horizon, 337 if target_horizon=="34w" else 323)
        ensemble_file="results/skills/ensemble_backward_stepwise_and_knn/skill-{}-{}-margin56-mean.h5".format(gt_id, target_horizon)
        backward_skills = pd.read_hdf(backward_file)
        knn_skills = pd.read_hdf(knn_file)
        ensemble_skills = pd.read_hdf(ensemble_file)
        backward_skills = backward_skills[backward_skills.start_date >= get_target_date("20170418", target_horizon)]
        knn_skills = knn_skills[knn_skills.start_date >= get_target_date("20170418", target_horizon)]
        ensemble_skills = ensemble_skills[ensemble_skills.start_date >= get_target_date("20170418", target_horizon)]
        skills = {'start_date': backward_skills.start_date,
                  'task': [task_str] * 26,
                  'backward': backward_skills.skill.tolist(),
                  'knn': knn_skills.skill.tolist(),
                  'ensemble': ensemble_skills.skill.tolist(),
                  'cfs': cfs_skills[task_str].tolist(),
                  'damped': damped_skills[task_str].tolist()}
        skills = pd.DataFrame(skills, index=None)
        all_skills = all_skills.append(skills, ignore_index=True)

In [ ]:
# Save dataframe of skills to csv. This gets used for data visualization: we make a histogram of contest-period skills 
# for each method and each task.
all_skills.to_csv("results/skills/all_skills_all_tasks_contest_year.csv")

In [ ]:
# Hardcoded average skill of top competitor in contest, from Forecast Rodeo website
top_competitor = {'temp34': 0.2855,
                  'temp56': 0.2357,
                  'precip34': 0.2144,
                  'precip56': 0.2162}

In [ ]:
# Create LaTeX table of average skills
print '\\begin{tabular}{l|ccccccc}\n\\toprule'
print 'task & multillr & autoknn & ensemble & contest debiased cfsv2 & damped & top competitor\\\\\n\\midrule'
for task in ["temp34", "temp56", "precip34", "precip56"]:
    task_str = (("temperature" if task.startswith("temp") else "precipitation") + ", " +
               ("weeks 3-4" if task.endswith("34") else "weeks 5-6"))
    task_skills = all_skills[all_skills.task==task]
    print '{} & {} & {} & {} & {} & {} & {}\\\\'.format(task_str,
                                                        round(task_skills["backward"].mean(), 4),
                                                        round(task_skills["knn"].mean(), 4),
                                                        round(task_skills["ensemble"].mean(), 4),
                                                        round(task_skills["cfs"].mean(), 4),
                                                        round(task_skills["damped"].mean(), 4),
                                                        top_competitor[task])
print '\\bottomrule\n\\end{tabular}'